# Introdução

Para o presente projeto, é necessário fazer o *download* os dados espaciais contendo a divisão municipal. Optou-se por baixar os dados do [**Instituto Brasileiro de Geografia e Estatistica (IBGE)**](https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais), em escala 1:250.000, suficiente para atender o objetivo do projeto, que trata-se de demostrar quais as divisões administrativas, por município, de diversos órgãos e/ou instituições.

Portanto, faz-se o *download* do arquivo zipado do **IBGE** e se descompacta o arquivo _shapefile_. Aproveitou-se para renomear o arquivo um nome mais inteligível.

[Repo SP](https://github.com/michelmetran/sp)

In [ ]:
import os
import requests

import pandas as pd
import geopandas as gpd

from osgeo import ogr
from zipfile import ZipFile

In [ ]:
#!pip install openpyxl

In [ ]:
# Create Directory
os.makedirs('data/tabs', exist_ok=True)
os.makedirs('data/shps', exist_ok=True)

## Functions

In [ ]:
def get_download(url, filename, path):
    # Define o nome do arquivo, com diretório, que será salvo
    zipfile = os.path.join(path, filename)

    # Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
    r = requests.get(url, stream=True)
    with open(zipfile, 'wb') as out:
        for chunk in r.iter_content(chunk_size=128):
            out.write(chunk)
    print('File "{}" download in "{}" directory.'.format(filename, path))

In [ ]:
def unzip(filename, path):
    # Get dir of zipfile
    zipfile = os.path.join(path, filename)
    path = os.path.dirname(zipfile)

    # Create a ZipFile Object
    with ZipFile(zipfile, 'r') as zip_obj:
        # Get a list of all archived file names from the zip
        list_files = []
        list_files = zip_obj.namelist()

        # Iterate over the file names
        for file in list_files:
            name, ext = os.path.splitext(file)
            zip_obj.extract(file, os.path.dirname(zipfile))
            os.rename(os.path.join(path, file), os.path.join(path, os.path.splitext(filename)[0]+ext))
            print('File "{}" extracted as "{}" in "{}" directory.'.format(file, os.path.splitext(filename)[0]+ext, path))

    # Remove file
    os.remove(zipfile)

# Dados Espaciais

## Limite Municipal 1:250.000

In [ ]:
# Input
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip'
filename = 'sp_ibge_250k.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
# Converte em *geojson*
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop(['NM_MUN', 'SIGLA_UF', 'AREA_KM2'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(columns={'CD_MUN': 'id_municipio'}, inplace=True)

# Ajusta Tipo
gdf['id_municipio'] = gdf['id_municipio'].astype(int)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Results
gdf

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
gdf_mun = pd.merge(
    df_mun,
    gdf,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
gdf_mun

In [ ]:
# Converte para Geodataframe
gdf_mun = gpd.GeoDataFrame(
    gdf_mun,
    crs='EPSG:4326',
)

# Salva Arquivo
gdf_mun.to_file(    
    os.path.join('data', 'shps', 'sp_250k_wgs84.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf_mun

## Limite Municipal 1:50.000

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/LimiteMunicipal/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=LimiteMunicipal'
filename = 'sp_igc_050k.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop(['Cod_Cetesb', 'Nome', 'Rotulo', 'UGRHI', 'Nome_ugrhi'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
    },
    inplace=True
)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Results
gdf

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
gdf = pd.merge(
    df_mun,
    gdf,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
gdf

In [ ]:
# Converte para Geodataframe
gdf = gpd.GeoDataFrame(
    gdf,
    crs='EPSG:4326',
)

# Salva Arquivo
gdf.to_file(
    os.path.join('data', 'shps', 'sp_050k_wgs84.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Dados Tabulares

## CETESB

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf

# Renomeia colunas
df.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
    },
    inplace=True
)

# Seleciona
df = df[['id_municipio', 'id_cetesb']].copy()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)


# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_idcetesb.csv'),
    index=False,
)

# Resultados
df.head()

## UGRHIs

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '/': ' / ',
        '  '  : ' ',
        ' De '  : ' de ',
        ' Da '  : ' da ',
        ' Do '  : ' do ',
        ' Dos '  : ' dos ',
        'Jundiai'  : 'Jundiaí',        
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf

# Renomeia colunas
df.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Seleciona
df = df[['id_municipio', 'id_ugrhi', 'nome_ugrhi']].copy()

# Renomeia Campos Coluna
df.loc[:, 'nome_ugrhi'] = df['nome_ugrhi'].astype(str).apply(lambda x: rename_nome(x))

# Resultados
df.head()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_ugrhi.csv'),
    index=False,
)

# Resultados
df.head()

## RMs e AUs

In [ ]:
# URL para baixar o arquivo zip
url = 'https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/municipios_por_regioes_metropolitanas/Situacao_2020a2029/Composicao_RMs_RIDEs_AglomUrbanas_2020_06_30.xlsx'

# Read File
df = pd.read_excel(
    url,
    engine='openpyxl',
)
print('Existem as seguintes colunas disponíveis:\n{}\n'.format(list(df.columns)))

df = df.loc[df['COD_UF'] == 35]
df = df[['COD_MUN', 'NOME', 'LEG', 'DATA']].copy()
df.reset_index(drop=True, inplace=True)

df.rename(
    columns={
        'COD_MUN': 'id_municipio',
        'NOME'   : 'nome_rm',
        'LEG'    : 'legislacao',
        'DATA'   : 'data',
    },
    inplace=True,
)

# Atribute Table
df['data_format'] = df['data'].dt.strftime('%d.%m.%Y')
df['legislacao_data'] = df[['legislacao', 'data_format']].apply(lambda x: ' de '.join(x), axis=1)
df.drop('data_format', axis=1, inplace=True)

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='right',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_rms.csv'),
    index=False,
)

# Resultados
df.head()

## Microregião

In [ ]:
# Lê geojson
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf[['CD_MUN']].copy()

# Renomeia colunas
df.rename(columns={'CD_MUN'  : 'id_municipio'}, inplace=True)

# Pega apenas os três primeiros caracteres de um campo de número de maneira mais eficiente
df['id_microregiao'] = [int(x[0:5]) for x in df['id_municipio']]

# Ajusta Tipo
df['id_municipio'] = df['id_municipio'].astype(int)

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_microregiao.csv'),
    index=False,
)

# Resultados
df.head()

## Mesoregião

In [ ]:
# Lê geojson
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Cria Cópia
df = gdf[['CD_MUN']].copy()

# Renomeia colunas
df.rename(columns={'CD_MUN'  : 'id_municipio'}, inplace=True)

# Pega apenas os três primeiros caracteres de um campo de número de maneira mais eficiente
df['id_mesoregiao'] = [int(x[0:4]) for x in df['id_municipio']]

# Ajusta Tipo
df['id_municipio'] = df['id_municipio'].astype(int)

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_mesoregiao.csv'),
    index=False,
)

# Resultados
df.head()

## Macrometropole

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/LimiteMacrometropole2015/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=LimiteMacrometropole2015'
filename = 'sp_macrometropole.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
# Lê arquivo
file = 'sp_macrometropole'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(columns={'Nome'  : 'macrometropole'}, inplace=True)

# Results
display(gdf.head())

# Cria Cópia
gdf_macro = gdf

In [ ]:
# Deleta Shapefile
file = 'sp_macrometropole'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
driver = ogr.GetDriverByName("ESRI Shapefile")

# Delete shapefile
if os.path.exists(shp):
    driver.DeleteDataSource(shp)

# Delete other files
for i in os.listdir(os.path.dirname(shp)):
    if i.split('.')[0] == file:
        os.remove(os.path.join('data', 'shps', i))

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
        'Nome'      : 'nome_municipio',
        'Rotulo'    : 'rotulo_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Deleta colunas
gdf.drop(['id_cetesb', 'nome_municipio', 'rotulo_municipio', 'nome_ugrhi', 'id_ugrhi'], axis=1, inplace=True)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_macro, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'macrometropole']].copy()

# Results
display(df.head())
gdf_inter.plot()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='right',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_macrometropole.csv'),
    index=False,
)

# Resultados
df.head()

## Regiões do Governo

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/RegioesGoverno/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=RegioesGoverno'
filename = 'sp_regioesgoverno.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        '  '  : ' ',
        ' De '  : ' de ',
        ' Da '  : ' da ',
        ' Do '  : ' do ',
        ' Dos '  : ' dos ',        
        ' Jose '  : ' José ',
        ' Joao '  : ' João ',
        'Jau'  : 'Jaú',
        'Sao ': 'São ',
        'Avare': 'Avaré',
        'Jundiai': 'Jundiaí',
        'Marilia': 'Marília',
        'Taubate': 'Taubaté',
        'Ribeirao': 'Ribeirão',
        'Guaratingueta': 'Guaratinguetá',
        'Fernandopolis': 'Fernandópolis',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Lê arquivo
file = 'sp_regioesgoverno'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(columns={'Nome'  : 'regiaogoverno_nome'}, inplace=True)

# Renomeia Campos Coluna
gdf.loc[:, 'regiaogoverno_nome'] = gdf['regiaogoverno_nome'].astype(str).apply(lambda x: rename_nome(x))

# Results
display(gdf.head())

# Cria Cópia
gdf_gov = gdf

In [ ]:
# Deleta Shapefile
file = 'sp_regioesgoverno'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
driver = ogr.GetDriverByName("ESRI Shapefile")

# Delete shapefile
if os.path.exists(shp):
    driver.DeleteDataSource(shp)

# Delete other files
for i in os.listdir(os.path.dirname(shp)):
    if i.split('.')[0] == file:
        os.remove(os.path.join('data', 'shps', i))

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
        'Nome'      : 'nome_municipio',
        'Rotulo'    : 'rotulo_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Deleta colunas
gdf.drop(['id_cetesb', 'nome_municipio', 'rotulo_municipio', 'nome_ugrhi', 'id_ugrhi'], axis=1, inplace=True)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_gov, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'regiaogoverno_nome']].copy()

# Results
display(df.head())
gdf_inter.plot()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_regiaogoverno.csv'),
    index=False,
)

# Resultados
df.head()

## Regiões Administrativas

In [ ]:
# Input
url = 'http://datageo.ambiente.sp.gov.br/geoserver/datageo/RegiaoAdministrativa/wfs?version=1.0.0&request=GetFeature&outputFormat=SHAPE-ZIP&typeName=RegiaoAdministrativa'
filename = 'sp_regioesadm.zip'
path = os.path.join('data', 'shps')

# Get Data
get_download(url, filename, path)
unzip(filename, path)

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        '  '  : ' ',
        ' De '  : ' de ',
        ' Da '  : ' da ',
        ' Do '  : ' do ',
        ' Dos '  : ' dos ',        
        ' Jose '  : ' José ',
        ' Joao '  : ' João ',
        'Jau'  : 'Jaú',
        'Sao ': 'São ',
        'Avare': 'Avaré',
        'Jundiai': 'Jundiaí',
        'Marilia': 'Marília',
        'Taubate': 'Taubaté',
        'Ribeirao': 'Ribeirão',
        'Guaratingueta': 'Guaratinguetá',
        'Fernandopolis': 'Fernandópolis',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Lê arquivo
file = 'sp_regioesadm'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Renomeia colunas
gdf.rename(columns={'Nome'  : 'regiaoadm_nome'}, inplace=True)

# Renomeia Campos Coluna
gdf.loc[:, 'regiaoadm_nome'] = gdf['regiaoadm_nome'].astype(str).apply(lambda x: rename_nome(x))

# Results
display(gdf.head())

# Cria Cópia
gdf_adm = gdf

In [ ]:
# Deleta Shapefile
file = 'sp_regioesadm'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
driver = ogr.GetDriverByName("ESRI Shapefile")

# Delete shapefile
if os.path.exists(shp):
    driver.DeleteDataSource(shp)

# Delete other files
for i in os.listdir(os.path.dirname(shp)):
    if i.split('.')[0] == file:
        os.remove(os.path.join('data', 'shps', i))

In [ ]:
# Lê arquivo
file = 'sp_igc_050k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Renomeia colunas
gdf.rename(
    columns={
        'Cod_ibge'  : 'id_municipio',
        'Cod_Cetesb': 'id_cetesb',
        'Nome'      : 'nome_municipio',
        'Rotulo'    : 'rotulo_municipio',
        'Nome_ugrhi': 'nome_ugrhi',
        'UGRHI'     : 'id_ugrhi',
    },
    inplace=True
)

# Deleta colunas
gdf.drop(['id_cetesb', 'nome_municipio', 'rotulo_municipio', 'nome_ugrhi', 'id_ugrhi'], axis=1, inplace=True)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Pega centroid
gdf_geometry = gdf.representative_point()
gdf['geometry'] = gdf_geometry

# Results
display(gdf.head())
gdf.plot()

In [ ]:
# Intersect
gdf_inter = gpd.sjoin(gdf, gdf_adm, how='inner', op='intersects')

# Convert to Dataframe
df = gdf_inter[['id_municipio', 'regiaoadm_nome']].copy()

# Results
display(df.head())
gdf_inter.plot()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Salva
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_regiaoadm.csv'),
    index=False,
)

# Resultados
df.head()

# Delete Temp Files

In [ ]:
# Deleta Shapefile
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
driver = ogr.GetDriverByName("ESRI Shapefile")

# Delete shapefile
if os.path.exists(shp):
    driver.DeleteDataSource(shp)

# Delete other files
for i in os.listdir(os.path.dirname(shp)):
    if i.split('.')[0] == file:
        os.remove(os.path.join('data', 'shps', i))

In [ ]:
# Deleta Shapefile
file = 'sp_igc_050k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
driver = ogr.GetDriverByName("ESRI Shapefile")

# Delete shapefile
if os.path.exists(shp):
    driver.DeleteDataSource(shp)

# Delete other files
for i in os.listdir(os.path.dirname(shp)):
    if i.split('.')[0] == file:
        os.remove(os.path.join('data', 'shps', i))